In [1]:
# importing necessary libraries
import pandas as pd 
import numpy as np
import requests
import json
import datetime
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm.notebook import tqdm
import time

# Billboard Weekly Hot 100 

In [2]:
top100_df = pd.read_csv('billboardHot100_1999-2019.csv')

In [3]:
top100_df.shape

(97225, 12)

creating year column

In [4]:
top100_df['year'] = pd.DatetimeIndex(top100_df['Week']).year

In [5]:
top100_df['year'].value_counts()

2017    5103
2018    5050
2009    5000
2016    4952
2012    4895
2004    4891
2010    4885
2011    4855
2006    4846
2013    4840
2014    4829
2003    4829
2000    4822
2015    4806
2008    4803
2005    4791
2001    4742
2002    4740
2007    4648
2019    2665
1999    2233
Name: year, dtype: int64

narrowing dataframe down to just be 2018-2019

In [6]:
top100_df['year_category'] = np.where(top100_df['year']>=2018,1,0)

In [7]:
top100_df['year_category'].value_counts()

0    89510
1     7715
Name: year_category, dtype: int64

In [144]:
df_billboard = top100_df[top100_df['year_category']==1]

In [145]:
df_billboard = df_billboard[['Artists','Name','Weekly.rank','Peak.position','Weeks.on.chart','Week','Date','Genre','Lyrics','Features']]

In [147]:
df_billboard.shape

(7715, 10)

# Spotify API

In [ ]:
def get_names():
    song_names = []
    for i in range(0,len(df)):
        name = df['Name'][i]
        song_names.append(name)
    return song_names
song_names = get_names()

In [14]:
# Spotify authentication for API calls
client_credentials_manager = SpotifyClientCredentials(client_id='930ca0608ac240ebb00350e101971444', client_secret='721f34eeee0746c49565619d88fcb6c5')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=4)

In [16]:
# creating list of artists 
def get_artists():
    artists = []
    for i in range(0,len(df)):
        artist = df['Artists'][i]
        artists.append(artist)
    return artists
list_artists = get_artists()
len(list_artists)

7715

In [18]:
# calling spotify API to get artist info 
def get_artist_info(list_artists):
    """Takes a list of artists and makes spotify API call"""
    results = []
    for artist in list_artists:
        results.append(sp.search(q=f'{artist}', type="artist", limit=1))
    return results
artist_info = get_artist_info(list_artists)

In [20]:
# creating list of song names 
def get_names():
    song_names = []
    for i in range(0,len(df)):
        name = df['Name'][i]
        song_names.append(name)
    return song_names
song_names = get_names()

In [21]:
# calling spotify API to get song info
def get_song_info(song_names):
    """Takes a list of artists and makes spotify API call"""
    results = []
    for song in song_names:
        results.append(sp.search(q=f'{song}', type="track", limit=1))
    return results
song_info = get_song_info(song_names)

In [23]:
song_info[0]['tracks']['items'][0]['id']

'2YpeDb67231RjR0MgVLzsG'

In [24]:
def get_ids():
    song_ids = []
    for i in range(0,len(song_info)): 
        song_id = song_info[i]['tracks']['items'][0]['id']
        song_ids.append(song_id)
    return song_ids
song_ids = get_ids()

In [27]:
# getting audio features using song ids 
audio_features = []
for thing in tqdm(song_ids):
    audio_features.append((sp.audio_features([f'{thing}'])))

In [149]:
# appending my dictionaries to a new list to create dataframe
list_of_dicts = []
for i in range(0,(len(audio_features))): 
    list_of_dicts.append(audio_features[i][0])

In [150]:
df_audio = pd.DataFrame(list_of_dicts)
df_audio.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.878,0.619,6,-5.560,1,0.1020,0.0533,0.00,0.1130,0.639,136.041,audio_features,2YpeDb67231RjR0MgVLzsG,spotify:track:2YpeDb67231RjR0MgVLzsG,https://api.spotify.com/v1/tracks/2YpeDb67231R...,https://api.spotify.com/v1/audio-analysis/2Ype...,157067,4
1,0.759,0.540,9,-6.039,0,0.0287,0.0370,0.00,0.0945,0.750,116.947,audio_features,0TK2YIli7K1leLovkQiNik,spotify:track:0TK2YIli7K1leLovkQiNik,https://api.spotify.com/v1/tracks/0TK2YIli7K1l...,https://api.spotify.com/v1/audio-analysis/0TK2...,190960,4
2,0.701,0.425,7,-10.965,1,0.3750,0.3280,0.13,0.1000,0.562,135.128,audio_features,2Fxmhks0bxGSBdJ92vM42m,spotify:track:2Fxmhks0bxGSBdJ92vM42m,https://api.spotify.com/v1/tracks/2Fxmhks0bxGS...,https://api.spotify.com/v1/audio-analysis/2Fxm...,194088,4
3,0.523,0.606,1,-4.754,0,0.0301,0.5120,0.00,0.1060,0.065,145.837,audio_features,161DnLWsx1i3u1JT05lzqU,spotify:track:161DnLWsx1i3u1JT05lzqU,https://api.spotify.com/v1/tracks/161DnLWsx1i3...,https://api.spotify.com/v1/audio-analysis/161D...,217867,4
4,0.798,0.675,6,-5.041,1,0.0442,0.0912,0.00,0.0894,0.842,101.956,audio_features,3HVWdVOQ0ZA45FuZGSfvns,spotify:track:3HVWdVOQ0ZA45FuZGSfvns,https://api.spotify.com/v1/tracks/3HVWdVOQ0ZA4...,https://api.spotify.com/v1/audio-analysis/3HVW...,219947,4


In [158]:
df_audio.to_csv('audio_features.csv')

In [152]:
df_billboard.head()

,Artists,Name,Weekly.rank,Peak.position,Weeks.on.chart,Week,Date,Genre,Lyrics,Features
0,"Lil Nas,",Old Town Road,1,1.0,7.0,2019-07-06,"April 5, 2019","Country,Atlanta,Alternative Country,Hip-Hop,Tr...","Old Town Road Remix \nOh, oh-oh\nOh\nYeah, I'm...",Billy Ray Cyrus
1,"Shawn Mendes, Camila Cabello",Senorita,2,NaN,NaN,2019-07-06,"June 21, 2019",Pop,Senorita \nI love it when you call me senorita...,NaN
2,Billie Eilish,Bad Guy,3,2.0,13.0,2019-07-06,"March 29, 2019","Hip-Hop,Dark Pop,House,Trap,Memes,Alternative ...","bad guy \nWhite shirt now red, my bloody nose\...",NaN
3,Khalid,Talk,4,3.0,20.0,2019-07-06,"February 7, 2019","Synth-Pop,Pop",Talk \nCan we just talk? Can we just talk?\nTa...,NaN
4,"Ed Sheeran, Justin Bieber",I Don't Care,5,2.0,7.0,2019-07-06,"May 10, 2019","Canada,UK,Dance,Dance-Pop,Pop",I Don't Care \nI'm at a party I don't wanna be...,NaN


In [153]:
# merging my 2 dataframes 
merged_df = df_billboard.merge(df_audio, left_index=True, right_index=True)
merged_df.head()

,Artists,Name,Weekly.rank,Peak.position,Weeks.on.chart,Week,Date,Genre,Lyrics,Features,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,"Lil Nas,",Old Town Road,1,1.0,7.0,2019-07-06,"April 5, 2019","Country,Atlanta,Alternative Country,Hip-Hop,Tr...","Old Town Road Remix \nOh, oh-oh\nOh\nYeah, I'm...",Billy Ray Cyrus,...,0.1130,0.639,136.041,audio_features,2YpeDb67231RjR0MgVLzsG,spotify:track:2YpeDb67231RjR0MgVLzsG,https://api.spotify.com/v1/tracks/2YpeDb67231R...,https://api.spotify.com/v1/audio-analysis/2Ype...,157067,4
1,"Shawn Mendes, Camila Cabello",Senorita,2,NaN,NaN,2019-07-06,"June 21, 2019",Pop,Senorita \nI love it when you call me senorita...,NaN,...,0.0945,0.750,116.947,audio_features,0TK2YIli7K1leLovkQiNik,spotify:track:0TK2YIli7K1leLovkQiNik,https://api.spotify.com/v1/tracks/0TK2YIli7K1l...,https://api.spotify.com/v1/audio-analysis/0TK2...,190960,4
2,Billie Eilish,Bad Guy,3,2.0,13.0,2019-07-06,"March 29, 2019","Hip-Hop,Dark Pop,House,Trap,Memes,Alternative ...","bad guy \nWhite shirt now red, my bloody nose\...",NaN,...,0.1000,0.562,135.128,audio_features,2Fxmhks0bxGSBdJ92vM42m,spotify:track:2Fxmhks0bxGSBdJ92vM42m,https://api.spotify.com/v1/tracks/2Fxmhks0bxGS...,https://api.spotify.com/v1/audio-analysis/2Fxm...,194088,4
3,Khalid,Talk,4,3.0,20.0,2019-07-06,"February 7, 2019","Synth-Pop,Pop",Talk \nCan we just talk? Can we just talk?\nTa...,NaN,...,0.1060,0.065,145.837,audio_features,161DnLWsx1i3u1JT05lzqU,spotify:track:161DnLWsx1i3u1JT05lzqU,https://api.spotify.com/v1/tracks/161DnLWsx1i3...,https://api.spotify.com/v1/audio-analysis/161D...,217867,4
4,"Ed Sheeran, Justin Bieber",I Don't Care,5,2.0,7.0,2019-07-06,"May 10, 2019","Canada,UK,Dance,Dance-Pop,Pop",I Don't Care \nI'm at a party I don't wanna be...,NaN,...,0.0894,0.842,101.956,audio_features,3HVWdVOQ0ZA45FuZGSfvns,spotify:track:3HVWdVOQ0ZA45FuZGSfvns,https://api.spotify.com/v1/tracks/3HVWdVOQ0ZA4...,https://api.spotify.com/v1/audio-analysis/3HVW...,219947,4


In [157]:
merged_df.to_csv('my_dataframe.csv')